# CSE 256 FA24: NLP UCSD PROJECT

In [53]:
from tqdm.notebook import tqdm
import pandas as pd
import os
import csv
import sys
import numpy as np
import time
import random
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
import textwrap
import torch

In [54]:
import os
os.environ['HF_HOME'] = 'D:\\Download\\UCSD\\cache'

In [55]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Using Device: {device}')

Using Device: cuda


In [56]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import tqdm

# Load Llama 1B and tokenizer
model_name = "meta-llama/Llama-3.2-1B"  # Using LLama 1B as base model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Ensure tokenizer has a padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use EOS token as PAD token
model = AutoModelForCausalLM.from_pretrained(model_name)

ds = load_dataset("stanfordnlp/imdb")
train_data = ds['train']
val_data = ds['validation']


# Veyr big dataset
# Load a sentiment dataset (example: SST2)
# ds = load_dataset("facebook/xnli", "all_languages")
# train_data = ds['train']
# val_data = ds['validation']

In [16]:
val_data['hypothesis'][0]

{'language': ['ar',
  'bg',
  'de',
  'el',
  'en',
  'es',
  'fr',
  'hi',
  'ru',
  'sw',
  'th',
  'tr',
  'ur',
  'vi',
  'zh'],
 'translation': ['اتصل بأمه حالما أوصلته حافلة المدرسية.',
  'Той се обади на майка си веднага щом училищният автобус го е оставил.',
  'Er rief seine Mutter an, sobald er aus dem Schulbus stieg.',
  'Τηλεφώνησε στη μαμά του μόλις το σχολικό λεωφορείο τον άφησε.',
  'He called his mom as soon as the school bus dropped him off.',
  'Llamó a su madre tan pronto como el autobús escolar lo dejó.',
  "Il a appelé sa mère dès que le bus scolaire l'a déposé.",
  'जब ही उसकी स्कूल बस ने उसे उतरा उसने अपनी माँ को बुलाया',
  'Он позвал маму, как только вышел из школьного автобуса.',
  'Alimwita mama yake mara tu basi ya shule ilipomshukisha.',
  'เขาโทรหาเเม่ของเขาอย่างรวดเร็วหลังจากที่รถโรงเรียนส่งเขาเเล้ว',
  'Okul servisi onu bırakır bırakmaz annesini aradı.',
  'اسنے اپنی امی کو فوران فون کیا جیسے ھی سکوول بس نے اسے اتارا',
  'Ngay khi xuống xe buýt của trường,

In [17]:
val_data['premise'][:2]

[{'ar': 'وقال، ماما، لقد عدت للمنزل.',
  'bg': 'И той каза: Мамо, у дома съм.',
  'de': 'und er hat gesagt, Mama ich bin daheim.',
  'el': 'Και είπε, Μαμά, έφτασα στο σπίτι.',
  'en': "And he said, Mama, I'm home.",
  'es': 'Y él dijo: Mamá, estoy en casa.',
  'fr': 'Et il a dit, maman, je suis à la maison.',
  'hi': 'और उसने कहा, माँ, मैं घर आया हूं।',
  'ru': 'И он сказал: Мама, я дома.',
  'sw': 'Naye akasema, Mama, niko nyumbani.',
  'th': 'และเขาพูดว่า, ม่าม๊า ผมอยู่บ้าน',
  'tr': 'Ve Anne, evdeyim dedi.',
  'ur': 'اور اس نے کہا امّی، میں گھر آگیا ہوں۔',
  'vi': 'Và anh ấy nói, Mẹ, con đã về nhà.',
  'zh': '他说，妈妈，我回来了。'},
 {'ar': 'وقال، ماما، لقد عدت للمنزل.',
  'bg': 'И той каза: Мамо, у дома съм.',
  'de': 'und er hat gesagt, Mama ich bin daheim.',
  'el': 'Και είπε, Μαμά, έφτασα στο σπίτι.',
  'en': "And he said, Mama, I'm home.",
  'es': 'Y él dijo: Mamá, estoy en casa.',
  'fr': 'Et il a dit, maman, je suis à la maison.',
  'hi': 'और उसने कहा, माँ, मैं घर आया हूं।',
  'ru': '

In [18]:
val_data['label'][0]

1

In [57]:
from transformers import DataCollatorWithPadding

# Define a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt", pad_to_multiple_of=8)

def preprocess_function(example):
    # Select language (e.g., 'en' for English)
      #   {'language': ['ar', # 'bg', # 'de', # 'el', # 'en', # 'es', # 'fr', # 'hi', # 'ru', # 'sw', # 'th', # 'tr', # 'ur', # 'vi', # 'zh']}
    language = 'en'

    # Extract premise and hypothesis in the selected language
    premise = example['premise'][language]
    hypothesis_index = example['hypothesis']['language'].index(language)
    hypothesis = example['hypothesis']['translation'][hypothesis_index]

    # Format as "<premise> </s> <hypothesis>"
    input_text = f"{premise} </s> {hypothesis}"

    # Tokenize the text
    tokenized = tokenizer(
        input_text,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # Add label
    tokenized['labels'] = example['label']
    return tokenized

# Tokenize train and validation datasets
train_dataset = train_data.map(preprocess_function, batched=False).with_format("torch")
val_dataset = val_data.map(preprocess_function, batched=False).with_format("torch")


Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

# LoRA Configuration

In [60]:
val_dataset[0]

{'premise': {'ar': 'وقال، ماما، لقد عدت للمنزل.',
  'bg': 'И той каза: Мамо, у дома съм.',
  'de': 'und er hat gesagt, Mama ich bin daheim.',
  'el': 'Και είπε, Μαμά, έφτασα στο σπίτι.',
  'en': "And he said, Mama, I'm home.",
  'es': 'Y él dijo: Mamá, estoy en casa.',
  'fr': 'Et il a dit, maman, je suis à la maison.',
  'hi': 'और उसने कहा, माँ, मैं घर आया हूं।',
  'ru': 'И он сказал: Мама, я дома.',
  'sw': 'Naye akasema, Mama, niko nyumbani.',
  'th': 'และเขาพูดว่า, ม่าม๊า ผมอยู่บ้าน',
  'tr': 'Ve Anne, evdeyim dedi.',
  'ur': 'اور اس نے کہا امّی، میں گھر آگیا ہوں۔',
  'vi': 'Và anh ấy nói, Mẹ, con đã về nhà.',
  'zh': '他说，妈妈，我回来了。'},
 'hypothesis': {'language': ['ar',
   'bg',
   'de',
   'el',
   'en',
   'es',
   'fr',
   'hi',
   'ru',
   'sw',
   'th',
   'tr',
   'ur',
   'vi',
   'zh'],
  'translation': ['اتصل بأمه حالما أوصلته حافلة المدرسية.',
   'Той се обади на майка си веднага щом училищният автобус го е оставил.',
   'Er rief seine Mutter an, sobald er aus dem Schulbus 

In [61]:
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments, Trainer

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

# Apply LoRA to model
model = get_peft_model(model, lora_config).to(device)

# Print trainable parameters
model.print_trainable_parameters()

# Define training arguments
training_args = TrainingArguments(
    output_dir="./lora_results",
    eval_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
)


trainable params: 851,968 || all params: 1,236,666,368 || trainable%: 0.0689


In [62]:
def custom_collate_fn(batch):
    print(f"Batch input_ids: {batch['input_ids'].shape}, labels: {batch['labels'].shape}")
    return data_collator(batch)

## Training LoRA here

In [63]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Train the model
trainer.train()

# Save LoRA fine-tuned model
model.save_pretrained("./lora_fine_tuned_llama")
tokenizer.save_pretrained("./lora_fine_tuned_llama")


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 14.80 GiB is allocated by PyTorch, and 18.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)